In [ ]:
import os, sys, re, pandas, collections
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as scs
import scipy as sp
import utils  # load_sheet, get_Ka, draw.
import importlib

import sameRiver
import sameRiver.adapter_fluorescence_utils as utils
import sameRiver.adapter_fluorescence

def hyperbolic(x, a, b):
    return (x*a)/(x+b)

def inverse_hyperbolic(x, a, b):
    return (-x*b)/(x-a)


In [ ]:
np.arange(start=0, stop=250)
plt.scatter()

In [ ]:
import sameRiver.adapter_fluorescence as af
importlib.reload(af)
importlib.reload(utils)

#df = utils.load_sheet(fname='180523.xlsx', sheet_name='180523_1')
#ip_control = df[df['Object']=='IP control'].copy()
#df, K = utils.get_Ka(df)

af.get_parameters_and_draw_staple(fname='180523.xlsx', sheet_name='180523_1')



In [ ]:
importlib.reload(af)
obj_color = {
    'αL3': '#DA1F26',
    'αL5': '#6ABD45',
    700: '#DF587A',
    800: '#74C6A0',
    '10 mM MgCl2': '#8DBFC6',
    '50 mM NaCl': '#5B8C88',
}

# File with the samples used for parameters.
df = af.load_sheet(fname='180522.xlsx', sheet_name='180522_1')

df = df[df['fmols']<100]
df = df[df['fmols']>3]

# Get fit and parameters
df = af.est_fmols_linear(df, staples=None)

l5 = df[df['Object']=='αL5']
l3 = df[df['Object']=='αL3']


def plot_est_vs_actual(xmax, filename):
    plt.clf()
    fig = plt.figure()
    
    plt.plot(
        l5['fmols'].tolist(), l5['Est. fmols'].tolist(), 'r.', markersize=14, alpha=0.3,
        markeredgewidth=0)
    plt.plot(
        l3['fmols'].tolist(), l3['Est. fmols'].tolist(), 'g.', markersize=14, alpha=0.3,
        markeredgewidth=0)
    
    plt.plot(range(0, xmax+50), range(0, xmax+50), 'k-', alpha=0.5)
    plt.plot(range(0, xmax+50), [x+3 for x in range(0, xmax+50)], 'k:', alpha=0.4)
    plt.plot(range(0, xmax+50), [x-3for x in range(0, xmax+50)], 'k:', alpha=0.4)

    plt.xlim(0, xmax)
    plt.ylim(0, xmax)
    fig.set_figwidth(5)
    fig.set_figheight(5)
    plt.xlabel('Approximate input fmols')
    plt.ylabel('Estimated fmols from gel shift')
    fig.savefig(filename)
    plt.show()
    plt.clf()


plot_est_vs_actual(15, './figs/est_vs_actual_fmols_known_l5l3_low_fmols.pdf')
plot_est_vs_actual(40, './figs/est_vs_actual_fmols_known_l5l3_50_fmols.pdf')

def errors(df):
    df['% error'] = [abs(100*(est-fmols)/fmols) for est, fmols in zip(
         df['Est. fmols'], df['fmols'])]
    df['fmols error'] = [abs(est-fmols) for est, fmols in zip(
         df['Est. fmols'], df['fmols'])]
    return df

l5 = errors(l5)
l3 = errors(l3)
#print(l5)
#print(l3)
_df = pandas.concat([l5, l3])
print(l5['% error'].mean())
print(l3['% error'].mean())

plt.clf()
fig = plt.figure()
sns.lmplot(
    data=_df, y='fmols error', x='fmols', hue='Object', scatter_kws={'alpha':0.3})
plt.show()
plt.clf()

fig = plt.figure()
sns.lmplot(
    data=_df, y='% error', x='fmols', hue='Object', scatter_kws={'alpha':0.3})
plt.show()
plt.clf()

def stripbox(_df, filename, xcol='', ycol='', hue=None, w=None, h=None):
    fig = plt.figure()
    ax = sns.boxplot(data=_df, x=xcol, y=ycol, #hue=hue,
                    fliersize=0, boxprops=dict(alpha=.2, linewidth=0),
                     #linewidth=0, #palette=obj_color,
                     #meanline=True, showmeans=True, 
                     #showmedians=False,
                    showfliers=False, showcaps=False, showbox=True, whis=0)
    sns.swarmplot(data=_df, y=ycol, x=xcol, alpha=0.5)
    if w is not None:
        fig.set_figwidth(w)
    if h is not None:
        fig.set_figheight(h)
    fig.savefig(filename)
    plt.show()
    plt.clf()
    
stripbox(_df, './figs/percent_error_known_L5L3_inputs.pdf',
         xcol='Object', ycol='% error', hue=None, w=1.2, h=3)

In [ ]:
"""

The development of an antisense oligo based method to quantify low femtomole amounts of adapter
at concentrations around 10 pg/ul, without using the adapter's fluorescence, necessitated some tinkering. 
First, we found fluorescence signal was more stable when dot blotted on nylon than on nitrocellulose (Fig Sx1), 
as we had done previously. Second, we noticed that when oligos were diluted to low pg/ul quantities,
the choice of diluent and blocking agent had very dramatic effects on the resulting signal (Fig Sx2).
We therefore systematically tested the effects of salt, carrier DNA, and PEG to retain fluorescence
and prevent sample loss from nonspecific adhesion (Fig Sx3). 
NaCl and MgCl2 had no major effect, and carrier DNA was insufficient to prevent fluorescence loss without
PEG400 or PEG8000 (Fig Sx3). PEG400 and PEG800 had similar effects, so we chose to use PEG400, which is
easier to pipette (Fig Sx3). We next found that a PEG400 concentration of 16.7% preserved DNA duplexes on a
native gel, while higher concentrations led to signal loss, possibly from duplex destabalization (Fig Sx4).

We also noted the fluorescence of both aL5 and aL3 decreases when tethered to the
same complex (Fig Sx5), presumably from the overlapping absorbance/emission spectra of the dyes resulting in 
signal interfernce when the dyes are in close proximity. We corrected for this effect and its relation
to incomplete labelling of the L3 adapter (see Methods).

We tested shifting known concentrations of L5 and L3 adapter and comparing estimates of L5/L3 amounts derived 
from the fluorescence of antisense oligos and the staple with the input fmols (Fig Sx6). 
Estimates from single replicates were almost always accurate to within 3 fmols in the 1-25 fmol range (Fig Sx6), 
and much of the error may be due to uncertainty in the input fmols. 
That is, since shifting our adapters at low fmol amounts
requires a large dilution from any concentration that can be measured at A260, as well as the fact
that low pg/ul quantities of DNA is liable to be lost to adhesion, the true input fmol amount is
only an estimate. 
Nevertheless the agreement between the expected and estimated amounts shows the method is reasonably accurate.



"""